This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](finite-difference.ipynb)

# Non-Uniform Finite Difference Splines

\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{1}{2} \left(
\frac{\boldsymbol{x}_i - \boldsymbol{x}_{i-1}}{t_i - t_{i-1}} +
\frac{\boldsymbol{x}_{i + 1} - \boldsymbol{x}_i}{t_{i + 1} - t_i}
\right)
\end{equation*}

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
t, t3, t4, t5, t6 = sp.symbols('t t3:7')

In [ ]:
control_values_FD = sp.Matrix([x3, x4, x5, x6])
control_values_FD

In [ ]:
xd4 = NamedExpression(
    'xdotbm4',
    ((x4 - x3) / (t4 - t3) + (x5 - x4) / (t5 - t4)) / 2)
xd4

In [ ]:
xd5 = NamedExpression(
    'xdotbm5',
    ((x5 - x4) / (t5 - t4) + (x6 - x5) / (t6 - t5)) / 2)
xd5

In [ ]:
control_values_H = sp.Matrix(sp.symbols('xbm4:6 xdotbm4:6'))
control_values_H

In [ ]:
M_H = NamedMatrix(
    r'{M_\text{H}}',
    sp.Matrix([[2, -2, 1, 1],
               [-3, 3, -2, -1],
               [0, 0, 1, 0],
               [1, 0, 0, 0]]) * sp.diag(1, 1, t5 - t4, t5 - t4))
M_H

In [ ]:
M_FDtoH = NamedMatrix(r'{M_{\text{FD$\to$H}}}', 4, 4)

In [ ]:
M_FD = NamedMatrix(r'{M_\text{FD}}', M_H.name * M_FDtoH.name)
M_FD

In [ ]:
NamedMatrix(control_values_H, M_FDtoH.name * control_values_FD)

In [ ]:
M_FDtoH.expr = sp.Matrix([
    [expr.expand().coeff(cv) for cv in control_values_FD]
    for expr in control_values_H.subs([xd4.args, xd5.args])])
M_FDtoH

In [ ]:
delta3, delta4, delta5 = sp.symbols('Delta3:6')
deltas = {
    t3: 0,
    t4: delta3,
    t5: delta3 + delta4,
    t6: delta3 + delta4 + delta5,
}

In [ ]:
M_FDtoH.subs(deltas)

In [ ]:
M_HtoFD = NamedMatrix(r'{M_\text{H$\to$FD}}', M_FDtoH.I.expr)
M_HtoFD.subs(deltas).expand()

In [ ]:
M_FD = M_FD.subs_symbols(M_H, M_FDtoH).doit()
M_FD.subs(deltas).expand()

In [ ]:
M_FD.subs(deltas).I.expand()

In [ ]:
uniform = {
    t3: 3,
    t4: 4,
    t5: 5,
    t6: 6,
    M_FD.name: sp.Symbol(r'{M_\text{FD,uniform}}'),
}

In [ ]:
M_FD.subs(uniform).pull_out(sp.S.Half)